# <center>CryptoCurrencies CA#2<center>
<h1><center>Transacting on Bitcoin TestNet / Firing up an Ethereum Node<center></h1>

#### <center>"Mohammad Sadegh - Aghili"<center>
#### <center>"810100274"<center>

### 1. Address Generation:

 1.1: Base58 and WIF address

In [12]:
# import sys

# !{sys.executable} -m pip install python-bitcoinlib


In [14]:
import os
import hashlib
import base58
from ecdsa import SigningKey, SECP256k1

In [16]:
# Generate a random 256-bit private key
private_key = os.urandom(32)
private_key_hex = private_key.hex()
print(f"Private Key (hex): {private_key_hex}")

# Add version byte for Testnet (0xEF) and convert to WIF
extended_key = b'\xef' + private_key
sha256_1 = hashlib.sha256(extended_key).digest()
sha256_2 = hashlib.sha256(sha256_1).digest()
checksum = sha256_2[:4]
wif = base58.b58encode(extended_key + checksum)
print(f"Private Key (WIF): {wif.decode()}")

# Derive the public key
sk = SigningKey.from_string(private_key, curve=SECP256k1)
vk = sk.get_verifying_key()
public_key = b'\x04' + vk.to_string()

# Generate the Bitcoin address (Testnet)
sha256_bpk = hashlib.sha256(public_key).digest()
ripemd160_bpk = hashlib.new('ripemd160', sha256_bpk).digest()
extended_ripemd160 = b'\x6f' + ripemd160_bpk
sha256_erpk = hashlib.sha256(extended_ripemd160).digest()
sha256_2_erpk = hashlib.sha256(sha256_erpk).digest()
checksum = sha256_2_erpk[:4]
binary_address = extended_ripemd160 + checksum
address = base58.b58encode(binary_address)
print(f"Bitcoin Address (Testnet): {address.decode()}")


Private Key (hex): 50751d06767b025b396fa191cfeefbdc350336ba76cd24009fd24aff24d66a22
Private Key (WIF): 92CMLkhwiSZ9abEVQZgScWySu9MK2cNC5qgzbjftXE3QekjgwXv
Bitcoin Address (Testnet): mhs9rZ8XmdsjZzsUCjdbNaCzFa2ZrjB7pP


### Explanation of Testnet vs Mainnet Addresses
* Testnet Addresses:
    * They start with m or n for P2PKH (Pay to Public Key Hash) addresses.
    * The version byte for WIF is 0xEF.
    * The version byte for the Bitcoin address is 0x6F.
* Mainnet Addresses:
    * They start with 1 for P2PKH addresses.
    * The version byte for WIF is 0x80.
    * The version byte for the Bitcoin address is 0x00.


1.2: Vanity Address Generator


In [ ]:
def generate_private_key():
    return os.urandom(32)

def private_key_to_wif(private_key):
    extended_key = b'\xef' + private_key
    sha256_1 = hashlib.sha256(extended_key).digest()
    sha256_2 = hashlib.sha256(sha256_1).digest()
    checksum = sha256_2[:4]
    return base58.b58encode(extended_key + checksum)

def private_key_to_public_key(private_key):
    sk = SigningKey.from_string(private_key, curve=SECP256k1)
    vk = sk.get_verifying_key()
    return b'\x04' + vk.to_string()

def public_key_to_address(public_key):
    sha256_bpk = hashlib.sha256(public_key).digest()
    ripemd160_bpk = hashlib.new('ripemd160', sha256_bpk).digest()
    extended_ripemd160 = b'\x6f' + ripemd160_bpk
    sha256_erpk = hashlib.sha256(extended_ripemd160).digest()
    sha256_2_erpk = hashlib.sha256(sha256_erpk).digest()
    checksum = sha256_2_erpk[:4]
    binary_address = extended_ripemd160 + checksum
    return base58.b58encode(binary_address)

def find_vanity_address(prefix):
    count = 0
    while True:
        private_key = generate_private_key()
        public_key = private_key_to_public_key(private_key)
        address = public_key_to_address(public_key)
        if address.decode()[1:4] == prefix:
            wif = private_key_to_wif(private_key)
            return private_key.hex(), wif.decode(), address.decode()
        count += 1
        if count % 1000 == 0:
            print(f"Tried {count} keys...")

# Example usage: Find an address with the prefix 'qwe'
desired_prefix = 'qwe'
private_key_hex, wif, address = find_vanity_address(desired_prefix)
print(f"Private Key (hex): {private_key_hex}")
print(f"Private Key (WIF): {wif}")
print(f"Bitcoin Address (Testnet): {address}")


### 2. Transacting on Bitcoin TestNet:

Here are some utility functions that you will need to use in order to make transactions. Fill the #TODO parts in the functions in order to be used in the next steps:

#### Utility Functions:

In [ ]:
import bitcoin.wallet
from bitcoin.core import COIN, b2lx, serialize, x, lx, b2x
from utils import *

bitcoin.SelectParams("testnet")  # Select the network (testnet or mainnet)
# Private key in WIF format XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
my_private_key = bitcoin.wallet.CBitcoinSecret("92CMLkhwiSZ9abEVQZgScWySu9MK2cNC5qgzbjftXE3QekjgwXv")
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)
destination_address = bitcoin.wallet.CBitcoinAddress('mjSk1Ny9spzU2fouzYgLqGUD8U41iR35QN')  # Destination address (recipient of the money)


def P2PKH_scriptPubKey(address):
    ######################################################################
    return [OP_DUP, OP_HASH160, Hash160(key), OP_EQUALVERIFY, OP_CHECKSIG]
    ######################################################################


def P2PKH_scriptSig(txin, txout, txin_scriptPubKey):
    ######################################################################
    ## Fill out the operations for P2PKH scriptSig                      ##
    signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, my_private_key)
    return [signature, my_public_key]
    ######################################################################


def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)

    txin_scriptPubKey = P2PKH_scriptPubKey(my_address)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig(txin, txout, txin_scriptPubKey)

    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(new_tx)


def run():
    ######################################################################
    amount_to_send = 0.1
    txid_to_spend = ('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')  # TxHash of UTXO
    utxo_index = 0  # UTXO index among transaction outputs
    ######################################################################

    print(my_address)  # Prints your address in base58
    print(my_public_key.hex())  # Print your public key in hex
    print(my_private_key.hex())  # Print your private key in hex
    txout_scriptPubKey = P2PKH_scriptPubKey(my_address)
    response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey)
    print(response.status_code, response.reason)
    print(response.text)  # Report the hash of transaction which is printed in this section result


In [13]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH


def send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin = create_txin(txid_to_spend, utxo_index)
    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)


def create_txin(txid, utxo_index):
    return CMutableTxIn(COutPoint(lx(txid), utxo_index))


def create_txout(amount, scriptPubKey):
    return CMutableTxOut(amount*COIN, CScript(scriptPubKey))


def create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, seckey):
    tx = CMutableTransaction([txin], [txout])
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return sig


def create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig):
    tx = CMutableTransaction([txin], [txout])
    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
    return tx


def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction,
    )


Now use the provided functions to make these transactions:

2.1: One input and two outputs transaction

2.2: Pay to MultiSig transaction (P2MS)


2.3: Custom Transaction (BitCoin Scripting)
